# Churn Rate Analysis
Esse notebook contempla a análise exploratória sobre a retenção de profissionais de medicina e enfermagem nas regiões de saúde com base em dados do CNES-PF.

## Carregar bibliotecas

In [ ]:
# Reload automático no Jupyter Notebook
%reload_ext autoreload
%autoreload 2

import traceback
import warnings
from dados import *
from tratamentos import *
from modelagem import *
from visualizacoes import *
from avaliacao import *

## Configurar bibliotecas

In [2]:
# Remover limite de exibição de linhas e colunas do pandas
pd.set_option('max_rows', 9999)
pd.set_option('max_columns', 9999)

# Evitar notações científicas no pandas
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Evitar warnings sobre perfomance no pandas
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

## Obter dados

In [3]:
df = obter_dados('profissionais.sql')
df.head()

,COMPETEN,CODUFMUN,categoria,CPF_PROF
0,201901,120020,Enfermeiro,<83><80>~{{<81>}}<81><83>
1,201901,120020,Médico,{<84><81>}}<81>{<82>
2,201901,120040,Enfermeiro,|~<83><82><83>{}}<82>}
3,201901,120040,Médico,{|<82><82><84>|}|~<84>
4,201901,120040,Médico,}}{{<80>{}<81><83>


In [4]:
geo = obter_dados('municipios.sql')
geo.head()

,uf,uf_sigla,cod_regsaud,regiao_saude,cod_municipiodv,cod_municipio,municipio
0,Rondônia,RO,11001,Vale do Jamari,1100452,110045,Buritis
1,Rondônia,RO,11003,Central,1100114,110011,Jaru
2,Rondônia,RO,11003,Central,1100254,110025,Presidente Médici
3,Rondônia,RO,11004,Madeira-Mamoré,1100205,110020,Porto Velho
4,Rondônia,RO,11004,Madeira-Mamoré,1100809,110080,Candeias do Jamari


In [5]:
# Realizar o join dos dados do CNES com tabela de municipios
df = df.merge(geo, left_on=['CODUFMUN'], right_on=['cod_municipio'])
df.head()

,COMPETEN,CODUFMUN,categoria,CPF_PROF,uf,uf_sigla,cod_regsaud,regiao_saude,cod_municipiodv,cod_municipio,municipio
0,201901,120020,Enfermeiro,<83><80>~{{<81>}}<81><83>,Acre,AC,12003,Juruá e Tarauacá/Envira,1200203,120020,Cruzeiro do Sul
1,201901,120020,Médico,{<84><81>}}<81>{<82>,Acre,AC,12003,Juruá e Tarauacá/Envira,1200203,120020,Cruzeiro do Sul
2,201901,120020,Médico,~<80>{<82>{<83>}{{,Acre,AC,12003,Juruá e Tarauacá/Envira,1200203,120020,Cruzeiro do Sul
3,201901,120020,Enfermeiro,<80><84><82>|<82><84>~<84>}<81><83>,Acre,AC,12003,Juruá e Tarauacá/Envira,1200203,120020,Cruzeiro do Sul
4,201901,120020,Enfermeiro,<81><84>{}<82>}|<80>,Acre,AC,12003,Juruá e Tarauacá/Envira,1200203,120020,Cruzeiro do Sul


## Processar os dados e criar os gráficos e arquivos CSV de retenção por região de saúde

In [6]:
# Realizar processamento para cada categoria profissional
for categoria in df['categoria'].unique():
    df_categoria = df.loc[df['categoria'] == categoria].copy()

    # Criar listas para salvar dados a serem exportados
    estatisticas_curvas = []
    triangulos = []
    mapes = []
    retencao_ano = []
    retencao_geral = []

    # Realizar o processamento separado para cada região de saúde
    i=0
    for regiao_saude in df['cod_regsaud'].unique():
        try:
            print(f'Processando {regiao_saude} - {i+1}/{len(df["cod_regsaud"].unique())}')
            df_categoria_regiao = df_categoria.loc[df_categoria['cod_regsaud'] == regiao_saude].copy()

            # Agregar os dados por cohorts
            cohort_pivot, cohort_sizes = gerar_triangulo_agregado(df_categoria_regiao.copy())

            # Obter o modelo chainladder
            cl_model = treinar_chainladder(cohort_pivot.copy())

            # Calcular profissionais restantes em cada período do cohort
            restantes = calcular_restantes(cl_model, cohort_pivot.copy())

            # Realizar a divisão de cada célula da tabela pivotada pelo tamanho do grupo para deixar em percentual
            percentual_restantes = calcular_percentual_restantes(restantes, cohort_sizes)

            # Calcular retenção anual (últimos 5 anos) por região de saúde
            retencao_anual = calcular_retencao_ano_regiao(percentual_restantes.copy(), regiao_saude)
            retencao_ano.append(retencao_anual)

            # Calcular retenção geral (média de todo o triangulo)
            retencao_geral.append(pd.DataFrame({'regiao_saude': [regiao_saude], 'retencao_geral': [cohort_pivot.divide(cohort_sizes, axis = 0).unstack().mean()]}))

            # Salvar dados do triangulo para exportar em csv
            cohort_pivot['cod_regsaud'] = regiao_saude
            triangulos.append(cohort_pivot.copy())
            cohort_pivot.drop(['cod_regsaud'], axis=1, inplace=True)

            # Criar features para ML sobre caracteristicas estatisticas da curva
            obs = gerar_ml_features(cohort_pivot, regiao_saude)
            estatisticas_curvas.append(obs)

            # Calcular mape
            mape = calcular_mape(cl_model).stack().mean()
            mapes.append(pd.DataFrame({'cod_regsaud': [regiao_saude], 'mape': [mape]}))

            # Gerar gráficos
            nome_regiao = df_categoria_regiao["regiao_saude"].values[0]
            uf_regiao = df_categoria_regiao["uf_sigla"].values[0]
            gerar_graficos(restantes.copy(), percentual_restantes.copy(), cl_model, f'Região de Saúde {nome_regiao} - {uf_regiao}', f'{categoria}{regiao_saude}')

            i+=1
        except Exception as e:
            print(f'Erro na região de saúde {regiao_saude}: {e}')
            # print(traceback.format_exc())
            continue


    # Exportar dados em CSV
    pd.concat(estatisticas_curvas).to_csv(f'csvs/{categoria}_estatisticas_curvas.csv', sep=';')
    pd.concat(triangulos).to_csv(f'csvs/{categoria}_triangulos.csv', sep=';')
    pd.concat(mapes).to_csv(f'csvs/{categoria}_mapes.csv', sep=';')
    pd.concat(retencao_ano).to_csv(f'csvs/{categoria}_retencao_ano.csv', sep=';')
    pd.concat(retencao_geral).to_csv(f'csvs/{categoria}_retencao_geral.csv', sep=';')

Processando 12003 - 1/450
Processando 12002 - 2/450
Processando 12001 - 3/450
Processando 27007 - 4/450
Processando 27008 - 5/450
Processando 27010 - 6/450
Erro na região de saúde 27010: '[180] not found in axis'
Processando 27001 - 6/450
Processando 27002 - 7/450
Processando 27003 - 8/450
Processando 27006 - 9/450
Processando 27005 - 10/450
Processando 27009 - 11/450
Processando 13008 - 12/450
Processando 13002 - 13/450
Processando 13006 - 14/450
Processando 13001 - 15/450
Processando 13009 - 16/450
Processando 13004 - 17/450
Processando 29025 - 18/450
Processando 29026 - 19/450
Processando 29022 - 20/450
Processando 29006 - 21/450
Processando 29024 - 22/450
Processando 29019 - 23/450
Processando 29003 - 24/450
Processando 29002 - 25/450
Processando 29012 - 26/450
Processando 29011 - 27/450
Processando 29015 - 28/450
Processando 29013 - 29/450
Processando 29007 - 30/450
Processando 29004 - 31/450
Processando 29027 - 32/450
Processando 29020 - 33/450
Processando 29001 - 34/450
Processa

C:\venvs\ufg_churn\lib\site-packages\chainladder\tails\base.py:120: RuntimeWarning: overflow encountered in exp
  sigma_ = xp.exp(time_pd * reg.slope_ + reg.intercept_)


Processando 51014 - 186/450
Processando 51016 - 187/450
Processando 51007 - 188/450
Processando 51003 - 189/450
Processando 25009 - 190/450
Processando 25002 - 191/450
Processando 25001 - 192/450
Processando 25016 - 193/450
Processando 25008 - 194/450
Processando 25007 - 195/450
Processando 25005 - 196/450
Processando 25012 - 197/450
Processando 25006 - 198/450
Processando 25004 - 199/450
Processando 25003 - 200/450
Processando 25015 - 201/450
Processando 25014 - 202/450
Processando 25010 - 203/450
Processando 26001 - 204/450
Processando 26003 - 205/450
Processando 26007 - 206/450
Processando 26002 - 207/450
Processando 26010 - 208/450
Processando 26004 - 209/450
Processando 26006 - 210/450
Processando 26008 - 211/450
Processando 26005 - 212/450
Processando 26009 - 213/450
Processando 26012 - 214/450
Processando 26011 - 215/450
Processando 22007 - 216/450


C:\venvs\ufg_churn\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\venvs\ufg_churn\lib\site-packages\chainladder\core\slice.py:250: RuntimeWarning: invalid value encountered in multiply
  obj.values = num_to_nan(obj.values * obj.get_array_module().array(key))


Erro na região de saúde 22007: Length of values (15) does not match length of index (16)
Processando 22003 - 216/450
Processando 22004 - 217/450
Processando 22005 - 218/450
Processando 22001 - 219/450
Processando 22011 - 220/450
Processando 22008 - 221/450
Processando 22002 - 222/450
Processando 22009 - 223/450
Processando 22006 - 224/450
Processando 33001 - 225/450
Processando 33002 - 226/450
Processando 33004 - 227/450
Processando 33005 - 228/450
Processando 33007 - 229/450
Processando 33009 - 230/450
Processando 33008 - 231/450
Processando 33003 - 232/450
Processando 33006 - 233/450
Processando 24004 - 234/450
Processando 24003 - 235/450
Processando 24001 - 236/450
Processando 24002 - 237/450
Processando 24007 - 238/450
Processando 24006 - 239/450
Processando 24008 - 240/450
Erro na região de saúde 24008: '[180] not found in axis'
Processando 24005 - 240/450
Processando 11001 - 241/450
Processando 11002 - 242/450
Processando 11006 - 243/450
Processando 11003 - 244/450
Processando 11

...

Processando 35113 - 439/450
Processando 35157 - 440/450
Processando 35152 - 441/450
Processando 35016 - 442/450
Processando 12003 - 1/450
Processando 12002 - 2/450
Processando 12001 - 3/450
Processando 27007 - 4/450
Processando 27008 - 5/450
Processando 27010 - 6/450
Processando 27001 - 7/450
Processando 27002 - 8/450
Processando 27003 - 9/450
Processando 27006 - 10/450
Processando 27005 - 11/450
Processando 27009 - 12/450
Processando 13008 - 13/450
Erro na região de saúde 13008: '[180] not found in axis'
Processando 13002 - 13/450
Processando 13006 - 14/450
Erro na região de saúde 13006: '[180] not found in axis'
Processando 13001 - 14/450
Processando 13009 - 15/450
Processando 13004 - 16/450
Processando 29025 - 17/450
Processando 29026 - 18/450
Processando 29022 - 19/450
Processando 29006 - 20/450
Processando 29024 - 21/450
Processando 29019 - 22/450
Processando 29003 - 23/450
Processando 29002 - 24/450
Processando 29012 - 25/450
Processando 29011 - 26/450
Processando 29015 - 27/450


C:\venvs\ufg_churn\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\venvs\ufg_churn\lib\site-packages\chainladder\core\slice.py:250: RuntimeWarning: invalid value encountered in multiply
  obj.values = num_to_nan(obj.values * obj.get_array_module().array(key))


Processando 21001 - 82/450
Processando 21007 - 83/450
Processando 21014 - 84/450
Processando 21003 - 85/450
Processando 21013 - 86/450
Processando 21019 - 87/450
Processando 21005 - 88/450
Processando 21006 - 89/450
Processando 21011 - 90/450
Processando 21012 - 91/450
Processando 21008 - 92/450
Processando 21004 - 93/450
Processando 21010 - 94/450
Processando 21017 - 95/450
Processando 21009 - 96/450
Processando 21015 - 97/450
Processando 21016 - 98/450
Processando 21018 - 99/450
Processando 21002 - 100/450
Processando 31062 - 101/450
Processando 31008 - 102/450
Processando 31041 - 103/450
Processando 31001 - 104/450
Processando 31092 - 105/450
Processando 31013 - 106/450
Processando 31006 - 107/450
Processando 31064 - 108/450
Processando 31075 - 109/450
Processando 31070 - 110/450
Processando 31087 - 111/450
Processando 31030 - 112/450
Processando 31021 - 113/450
Processando 31016 - 114/450
Processando 31017 - 115/450
Processando 31034 - 116/450
Processando 31048 - 117/450
Processand

C:\venvs\ufg_churn\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\venvs\ufg_churn\lib\site-packages\chainladder\core\slice.py:250: RuntimeWarning: invalid value encountered in multiply
  obj.values = num_to_nan(obj.values * obj.get_array_module().array(key))


Processando 15001 - 297/450
Processando 15010 - 298/450
Processando 15007 - 299/450
Processando 41016 - 300/450
Processando 41002 - 301/450
Processando 41020 - 302/450
Processando 41011 - 303/450
Processando 41003 - 304/450
Processando 41010 - 305/450
Processando 41013 - 306/450
Processando 41012 - 307/450
Processando 41021 - 308/450
Processando 41004 - 309/450
Processando 41015 - 310/450
Processando 41009 - 311/450
Processando 41008 - 312/450
Processando 41014 - 313/450
Processando 41005 - 314/450
Processando 41001 - 315/450
Processando 41017 - 316/450
Processando 41022 - 317/450
Processando 41019 - 318/450
Processando 41007 - 319/450
Processando 41006 - 320/450
Processando 41018 - 321/450
Processando 43003 - 322/450
Processando 43010 - 323/450
Processando 43029 - 324/450
Processando 43004 - 325/450
Processando 43027 - 326/450
Processando 43021 - 327/450
Processando 43019 - 328/450
Processando 43022 - 329/450
Processando 43005 - 330/450
Processando 43008 - 331/450
Processando 43009 - 